In [3]:
import pandas as pd
import os
import weaviate
from openai import OpenAI
from weaviate.classes.config import Configure
import weaviate.classes as wvc
from langchain_community.document_loaders import BSHTMLLoader
from typing import Dict, List, Optional
from dataclasses import dataclass
from uuid import uuid4
from dataclasses import dataclass
import re
from typing import List


@dataclass
class TextSection:
    id: str
    content: str
    type: str  # 'skyrius', 'straipsnis', or 'paragraph'
    number: str  # Chapter number, article number, or paragraph number
    type_text: str  # "SKYRIUS", "straipsnis", etc.
    title: str  # Full title or name of the section
    heading: Optional[str] = None  # Additional heading/description (e.g., "BENDROSIOS NUOSTATOS")
    parent_id: Optional[str] = None


html_files_dir = 'vmi_docs'
collection_name = 'Vmi_docs'
csv_file = 'vmi_docs.csv'
wcd_url = 'https://xh1j9trzu5cervreztxw.c0.europe-west3.gcp.weaviate.cloud'
wcd_api_key = 'uTTyayyrfwyn98zBq6ukAcIAVnEJjkBWMLac'
openai_api_key = os.environ["OPENAI_API_KEY"]

weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=wcd_url,
    auth_credentials=wvc.init.Auth.api_key(wcd_api_key),
    headers={"X-OpenAI-Api-Key": openai_api_key}
)

client = OpenAI()


In [28]:
@dataclass
class Chapter:
    id: str
    number: str  # Roman numeral
    type: str  # "SKYRIUS"
    title: str  # Chapter title
    content: str  # Full content
    raw_header: str  # Original header text


def split_into_chapters(text: str) -> List[Chapter]:
    """
    Split text into chapters based on 'SKYRIUS' headers.
    Returns list of Chapter objects.

    Example input:
    I SKYRIUS
    BENDROSIOS NUOSTATOS
    [chapter content...]
    II SKYRIUS
    KITOS NUOSTATOS
    [chapter content...]
    """
    # Pattern to match Roman numerals followed by SKYRIUS and title
    chapter_pattern = r'([IVX]+)\s*SKYRIUS\s*\n\s*(?:\*\*)?([^*\n]+)(?:\*\*)?\s*\n'

    # Find all chapter starts
    chapter_starts = list(re.finditer(chapter_pattern, text, re.MULTILINE))
    chapters = []

    # Process each chapter
    for i, match in enumerate(chapter_starts):
        chapter_id = str(uuid4())
        chapter_number = match.group(1)
        chapter_title = match.group(2).strip()
        chapter_header = match.group(0)

        # Calculate chapter content (from this chapter start to next chapter start or end of text)
        content_start = match.end()
        if i < len(chapter_starts) - 1:
            content_end = chapter_starts[i + 1].start()
        else:
            content_end = len(text)

        chapter_content = text[content_start:content_end].strip()

        chapter = Chapter(
            id=chapter_id,
            number=chapter_number,
            type="SKYRIUS",
            title=chapter_title,
            content=chapter_content,
            raw_header=chapter_header.strip()
        )
        chapters.append(chapter)

    return chapters


Chapter #I
Title: BENDROSIOS NUOSTATOS
Content preview: 1 straipsnis. Įstatymo paskirtis
Some content here......
--------------------------------------------------

Chapter #II
Title: KITOS NUOSTATOS
Content preview: 2 straipsnis. Kiti dalykai
More content here......
--------------------------------------------------


In [173]:
def clean_text(text: str) -> str:
    """
    Clean text by removing article adjustments, end document text, and normalize whitespace.
    """
    # Define patterns to remove
    amendment_pattern = r'(?:Papildyta straipsnio punktu:|Straipsnio dalies pakeitimai:|' \
                        r'Papildyta straipsniu:|Papildyta straipsnio dalimi:|' \
                        r'Straipsnio dalies numeracijos pakeitimas:|Straipsnio pakeitimai:)' \
                        r'\s+(?:Nr\.[^\n]+(?:\([^\)]+\))?\s*)+'
    cleaned_text = re.sub(amendment_pattern, '', text, flags=re.MULTILINE)

    # Remove all adjustment patterns

    # Remove everything after the end marker
    end_marker = "Skelbiu šį Lietuvos Respublikos Seimo priimtą įstatymą."
    if end_marker in cleaned_text:
        cleaned_text = cleaned_text.split(end_marker)[0]

    # # Remove double whitespaces and normalize newlines
    cleaned_text = re.sub(r'[ ]+', ' ', cleaned_text)  # Replace multiple spaces with single space
    cleaned_text = re.sub(r'\n+', ' \n', cleaned_text)  # Normalize multiple newlines
    cleaned_text = cleaned_text.strip()

    return cleaned_text

In [86]:
from dataclasses import dataclass
import re
from typing import List
from uuid import uuid4


@dataclass
class Article:
    id: str
    number: str  # Article number
    title: str  # Article title
    content: str  # Full content
    raw_header: str  # Original header text


def split_into_articles(text: str) -> List[Article]:
    """
    Split text into articles based on "straipsnis" headers.
    Example: "1 straipsnis. Įstatymo paskirtis ir taikymo sritis"
    """
    # Pattern to find article headers with lookahead for next article or end of string
    article_pattern = r'(\d+)\s+straipsnis\.\s*([^\n]+)'

    # Find all matches of article headers
    matches = list(re.finditer(article_pattern, text))
    articles = []

    for i in range(len(matches)):
        current_match = matches[i]
        article_id = str(uuid4())
        article_number = current_match.group(1)
        article_title = current_match.group(2).strip()
        article_header = current_match.group(0)

        # Get content until next article or end of text
        start_pos = current_match.end()
        if i < len(matches) - 1:
            end_pos = matches[i + 1].start()
        else:
            end_pos = len(text)

        # Extract content
        content = text[start_pos:end_pos].strip()

        # Create Article object
        article = Article(
            id=article_id,
            number=article_number,
            title=article_title,
            content=content,
            raw_header=article_header
        )
        articles.append(article)

    return articles



In [4]:
df = pd.read_csv(csv_file)[0:1]

In [6]:
for idx, row in df.iterrows():
    metadata = {
        "name": str(row['name']),
        "valid_from": str(row['valid_from']),
        "valid_to": str(row['valid_to']),
        "doc_link": str(row['doc_link']),
        "doc_id": str(row['id']),
        "version": str(row['version']),
        "url": str(row['url'])
    }



In [167]:
input_dir = 'vmi_docs'
file_path = os.path.join(input_dir, f"{metadata['version']}.html")
html_loader = BSHTMLLoader(file_path, bs_kwargs={'features': 'html.parser'})
documents = html_loader.load()
content = "\n".join([doc.page_content for doc in documents])

In [169]:
result = split_into_chapters(content)
print(result[3].content)

16 straipsnis. Apmokestinamųjų pajamų apskaičiavimo tvarka


1. Jeigu šiame straipsnyje nenustatyta kitaip, apskaičiuojant apmokestinamąsias pajamas, iš pajamų šio Įstatymo nustatyta tvarka atimama:


1) šio Įstatymo 17 straipsnyje nurodytos neapmokestinamosios pajamos; 
Straipsnio punkto pakeitimai:
Nr. XIII-841, 2017-12-07, paskelbta TAR 2017-12-20, i. k. 2017-20568
 




2) pajamos, apmokestintos įsigijus verslo liudijimą;




3) leidžiami atskaitymai, susiję su individualios veiklos pajamų gavimu arba uždirbimu, – šio Įstatymo 18 straipsnyje nustatyta tvarka;




4) per mokestinį laikotarpį parduoto ar kitaip perleisto nuosavybėn ne individualios veiklos turto, išskyrus finansinius produktus, įsigytus iš investicinės sąskaitos lėšų, ir individualios veiklos turtui priskirto nekilnojamojo pagal prigimtį daikto įsigijimo kaina ir su to turto ar daikto pardavimu ar kitokiu perleidimu nuosavybėn susijusios išlaidos – šio Įstatymo 19 straipsnyje nustatyta tvarka; 
Straipsnio punkto pake

In [207]:
def parse_document(content:str) -> List:
    chapters_list = split_into_chapters(content)
    # print(chapters_list[1])
    articles = []
    for chapter in chapters_list:
        cleaned_chapter_text = clean_text(chapter.content)
        chapter_articles = split_into_articles(cleaned_chapter_text)

        for article in chapter_articles:
            print(f"  Article {article.number}: {article.title}")
            articles.append({
                "chapter_id": chapter.id,
                "chapter_number": chapter.number,
                "chapter_type": chapter.type,
                "chapter_title": chapter.title,
                "chapter_raw_header": chapter.raw_header,
                "article_id": article.id,
                "article_number": article.number,
                "article_title": article.title,
                "article_content": article.content,
                "article_raw_header": article.raw_header
            })

    return articles

In [208]:
articles = parse_document(content)

  Article 1: Įstatymo paskirtis ir taikymo sritis
  Article 2: Pagrindinės Įstatymo sąvokos
  Article 3: Pajamų mokesčio mokėtojai
  Article 4: Nuolatinis Lietuvos gyventojas
  Article 5: Pajamų mokesčio objektas
  Article 6: Pajamų mokesčio tarifai
  Article 7: Mokestinis laikotarpis
  Article 8: Pajamų pripažinimas
  Article 9: Pajamos natūra
  Article 10: Individualios veiklos pajamos
  Article 11: Pajamos, gautos vieneto likvidavimo atveju
  Article 12: Pajamos iš paskirstytojo pelno
  Article 121: Per investicinę sąskaitą gautos pajamos
  Article 13: Pozityviosios pajamos
  Article 14: Jūrininkų pajamų, gautų už darbą laivo reiso metu, nustatymas
  Article 15: Sandorių arba ūkinių operacijų vertės koregavimas ir pajamų ar išmokų apibūdinimas iš naujo
  Article 16: Apmokestinamųjų pajamų apskaičiavimo tvarka
  Article 17: Neapmokestinamosios pajamos
  Article 18: Leidžiami atskaitymai, susiję su individualios veiklos pajamų gavimu arba uždirbimu
  Article 181: Mokestinio laikotarpi

In [212]:
articles[0].keys()

dict_keys(['chapter_id', 'chapter_number', 'chapter_type', 'chapter_title', 'chapter_raw_header', 'article_id', 'article_number', 'article_title', 'article_content', 'article_raw_header'])